In [1]:
from web3 import Web3, IPCProvider

In [2]:
w3 = Web3(IPCProvider('/shakespeare/sandipan/Ethereum/.ethereum/geth.ipc'))

In [3]:
w3.eth.getBlock('latest')

AttributeDict({'difficulty': 1404196117668992,
 'extraData': HexBytes('0x65746865726d696e652d657536'),
 'gasLimit': 6712392,
 'gasUsed': 1511793,
 'hash': HexBytes('0x5aac599ec2dc25ef4eea8f7c32561508618e0e46684edc9296a6850ed1e693f0'),
 'logsBloom': HexBytes('0x00000000000000000000000040000002000000000000000004140000000000000000000020000000000000000000000000000000000000020000002000000000000000000000000000000008000000000400000000000000000100000000000000000000000000000100004080000000000000000000000000000010000000000000020000000000000000000080000000000000000200000000002000000100000000000000000000000800000001000000000000000000000000004000000000000002000000000000000000000000000000000020080020000000000000000000000000000000000000000000000000001000400000000000000000002000'),
 'miner': '0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8',
 'mixHash': HexBytes('0x1d26902a6308f8dcd0c75a88e9f9d8ab30c615053475c35ac2c01a9e2bedcae6'),
 'nonce': HexBytes('0xf4dea40c0297a887'),
 'number': 4521390,
 'parentHash'

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('transaction_till_100000.csv')

In [3]:
df.head()

,From,To,Value,Time,Block
0,ethereum,0,5.000000e+18,0,0
1,ethereum,05a56e2d52c817161883f50c441c3228cfe54d9f,5.000000e+18,1438269988,1
2,ethereum,0193d941b50d91be6567c7ee1c0fe7af498b4137,5.000000e+18,1438270128,10
3,ethereum,e6a7a1d47ff21b6321162aea7c6cb457d5476bca,5.000000e+18,1439799138,100000
4,ethereum,9beb4d346c6309a9ce4f27393c6322c8f03a3be6,5.000000e+18,1438336146,10099


In [17]:
add = df.From.unique()[1]


In [20]:
add = Web3.toChecksumAddress(add)

In [22]:
w3.eth.getBalance(add,100)

140000000000000000000

In [26]:
df_b = df[df['Block']==1000][['From','To']]
    

In [34]:
import numpy as np

In [35]:
np.concatenate(df_b.From.unique(),df_b.To.unique())

TypeError: only integer scalar arrays can be converted to a scalar index

In [38]:
address = np.append(df_b.From.unique(),df_b.To.unique())

In [41]:
blockNum = 100
for i in range(len(address)):
    if address[i]!='ethereum':
        account = Web3.toChecksumAddress(address[i])
        balance = w3.eth.getBalance(account,blockNum)
        print(address[i],balance)

bb7b8287f3f0a933474a79eae42cbca977791171 151250000000000000000


In [81]:
def fillBalanceTable(blockNum,fname):
    df_b = df[df['Block']==blockNum][['From','To']]
    address = np.append(df_b.From.unique(),df_b.To.unique())
    #with open(fname,'a') as ft:
    for i in range(len(address)):
        if address[i]!='ethereum' and address[i]!='0':
            account = Web3.toChecksumAddress(address[i])
            balance = w3.eth.getBalance(account,blockNum)
            ft = open(fname,'a')
            ft.write(account+','+str(balance)+','+str(blockNum))
            ft.write('\n')
            ft.close()

In [49]:
fillBalanceTable(10000)

bf71642d7cbae8faf1cfdc6c1c48fcb45b15ed22 147343750000000000000


In [47]:
df[df.Block==10000]

,From,To,Value,Time,Block
103,ethereum,bf71642d7cbae8faf1cfdc6c1c48fcb45b15ed22,5.000000e+18,1438334627,10000


In [74]:
fs = open('blockAccountBalance1.csv','w')
fs.write('Account,Balance,Block\n')
fs.close()

In [57]:
for i in range(1,1000):
    fillBalanceTable(i,'blockAccountBalance.csv')

In [62]:
df_1 = pd.read_csv('blockAccountBalance.csv')

In [63]:
df_1.head()

,Account,Balance,Block
0,0x05a56E2D52c817161883f50c441c3228CFe54d9f,5000000000000000000,1
1,0xdd2F1e6e498202e86D8f5442Af596580A4f03c2C,5000000000000000000,2
2,0x5088D623ba0fcf0131E0897a91734A4D83596AA0,5156250000000000000,3
3,0xc8ebccC5F5689Fa8659d83713341e5aD19349448,3750000000000000000,3
4,0xc8ebccC5F5689Fa8659d83713341e5aD19349448,8906250000000000000,4


In [64]:
df_1.Block.max()

5000

In [61]:
for i in range(1000,5001):
    fillBalanceTable(i,'blockAccountBalance.csv')

In [82]:
def createBalanceCSV(start,end,fname):
    for i in range(start,end+1):
        fillBalanceTable(i,fname)
        print('finished for block: ',i)

In [67]:
from multiprocessing import Process

In [77]:
jobs = []

In [83]:
jobs.append(Process(target=createBalanceCSV, args=(5305,7500,"blockAccountBalance.csv")))
jobs.append(Process(target=createBalanceCSV, args=(7501,10000,"blockAccountBalance.csv")))
jobs.append(Process(target=createBalanceCSV, args=(10001,12500,"blockAccountBalance.csv")))
jobs.append(Process(target=createBalanceCSV, args=(12501,15000,"blockAccountBalance.csv")))

In [84]:
for j in jobs:
    j.start()
    for j in jobs:
        j.join()

AssertionError: cannot start a process twice

In [80]:
df[df.Block==5305]

,From,To,Value,Time,Block
51641,ethereum,0,5.000000e+18,1438288819,5305


In [45]:
import numpy as np
def balanceAccountBlock(blockNum, df): # retrieve balances of all accounts given a block number
    df_b = df[df['Block']<=blockNum][['From','To']]
    address = np.append(df_b.From.unique(),df_b.To.unique())
    address = list(set(address))
    print("addresses to look into: ",len(address))
    with open('AccountBalanace'+str(blockNum)+'.csv','w') as ft:
        ft.write('Account,Balance,Block')
        ft.write('\n')
        for add in address:
            if add!='ethereum' and add!='0':
                try:
                    account = Web3.toChecksumAddress(add)
                    balance = w3.eth.getBalance(account,blockNum)
                    ft.write(add+','+str(balance)+','+str(blockNum))
                    ft.write('\n')
                except:
                    print(add)
    

In [46]:
balanceAccountBlock(100000,df)

addresses to look into:  7880
b67b6bb30bb68ed96f23f1fb8c6da01b3c37f6d4
e706a05a40761915bc70310f7a94e6b2b596dbf8
6.12416344436736E+039
c47d9e6c75d388aeb92574429dad43a148f2f3ea
164334243ea25246e0703f3bfca824a30e87197f
288e4356f12d0d3d0f8df3506e457e7cfd5f4ab0
87fd1c6dfa355a18911997a9bffa561272829453
5a192b964afd80773e5f5eda6a56f14e25e0c6f3
3da4225ad67e0e5729ff1beaa10bb6e41ddf31f7
8f1c0dd1d8d230f7557b63967586eac9f139dcee
e7ca5a8e75ef17f845c14a85c50e39b0dd30b8b7
710c5febe84c40b3c12be2cb2378b863f67235df
426f78f70db259ac8534145b2934f4ef1098b5d8
9ff6094d95665aa596f05f1f4841d9b29df75188
91409b1271111e5f3a526e53031d7553ec8959d3
c54671b79ffcdc38c550bfc2f43aca4faf8d6f64
f2ae859397af92154241ead0ac7c4ce3bea76e8f
95227f88247580d7c3927bdafe78e73a675cdc76
afa919981af838667228c343c15d7fde1bfa211c
3899a1cdb9b4275d84860c6ab72d09e08dcc589a
ec6a6bd0df7186d3c2fce7c46264b0b9b2e5e1e5
501e496ad1c7fa8d0eed694450ba29b1ec0783bf
ed1f560d21dde85fac18f23530c257c572f2f5ee
d9a6ffe60b9c5e07f3d48a6880e3a2ee4659cdb2
a725d

In [58]:
account = Web3.toChecksumAddress('c47d9e6c75d388aeb92574429dad43a148f2f3ea')

In [59]:
w3.eth.getBalance(account,100000)

31307600945853274600

In [52]:
df_balance = pd.read_csv('AccountBalanace100000.csv')

In [53]:
df_balance.head()

,Account,Balance,Block
0,32728d6184d9a6abd2541294d4cbde925924291a,0,100000
1,9f29a11e58479e87f4d10413ec0cf7f3ff34c4c1,240625000000000000000,100000
2,632bb14025ba69a50f8c99f962998f06b70f76b4,981800000000000000,100000
3,f79650c3c0a99fa165c62bbeab819e706182c10f,0,100000
4,7705158b409f54a0e4373a31bbdc2a7abd8161ae,0,100000


In [24]:
x = Web3.fromWei(25000000000000,'ether')

In [26]:
float(x)

2.5e-05

In [30]:
import pandas as pd

In [56]:
df_balance.Balance = df_balance.Balance.apply(lambda x: float(x))

In [21]:
df[df.To=='6.12416344436736E+039']

,From,To,Value,Time,Block
91311,7744774422cd8b2d5895d7bc552b9a57388db228,6.12416344436736E+039,1.000000e+18,1439452895,78378
91618,7744774422cd8b2d5895d7bc552b9a57388db228,6.12416344436736E+039,4.964882e+20,1439455427,78550


In [31]:
df_balance.describe()

,Balance,Block
count,1.262400e+04,12624.0
mean,2.408559e+21,100000.0
std,9.523821e+22,0.0
min,0.000000e+00,100000.0
25%,0.000000e+00,100000.0
50%,1.186848e+13,100000.0
75%,5.428642e+18,100000.0
max,8.809482e+24,100000.0


In [36]:
df_balance.Balance.min()

0.0

In [38]:
df_balance.Balance.max()

8.8094822394799999e+24

In [39]:
balance = df_balance.Balance.sort_values()

In [40]:
balance.head()

9100    0.0
3169    0.0
3168    0.0
3167    0.0
3166    0.0
Name: Balance, dtype: float64

In [41]:
balance.tail()

1181    2.450000e+24
7545    2.450000e+24
1391    3.454732e+24
6882    3.454732e+24
8460    8.809482e+24
Name: Balance, dtype: float64

In [43]:
G_coeff = 0.0
balance.iloc[0]

0.0

In [44]:
n = balance.count()

In [45]:
n

12624

In [50]:
balance.iloc[12623]

8.8094822394799999e+24

In [51]:
b = balance.values

In [52]:
b

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         3.45473167e+24,   3.45473167e+24,   8.80948224e+24])

In [53]:
b.size

12624

In [54]:
b.cumsum()

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         1.81414353e+25,   2.15961670e+25,   3.04056492e+25])

In [55]:
import matplotlib.pyplot as plt

In [58]:
a = np.array([1,2,3,4,5,6])

In [59]:
b.sum()

3.040564923603721e+25

In [66]:
A = [1,2,3,4,5]
n = len(A)+1
c = map(lambda(i,x):i*x, enumerate(A))

SyntaxError: invalid syntax (<ipython-input-66-529ddbefa042>, line 3)

In [175]:
a = [0, 0, 0, 0, 1]
a.sort()
n = len(a)
num = sum(list(map(lambda x: (n-x[0])*x[1], enumerate(a))))
den = sum(a)

In [176]:
(n+1 - 2*(num/den))/n

0.8

In [81]:
n = b.size
answer = list(map(lambda x: (n-x[0])*x[1], enumerate(b)))

In [86]:
a = np.array(a)
for i,j in enumerate(a):
    print(i,j)

0 1
1 3
2 5
3 6
4 8


In [83]:
(n+1-2*sum(answer)/b.sum())/n

0.99439717617577084

In [54]:
def giniCoefficient(df_balance): # balance is pandas table....
    balance = df_balance.Balance.sort_values()
    b = balance.values # convert to a numpy array
    n = b.size
    num = sum(list(map(lambda x: (n-x[0])*x[1], enumerate(b))))
    den = b.sum()
    return (n+1-2*(num/den))/n

In [57]:
giniCoefficient(df_balance)

0.99237693778659386

In [5]:
df.describe()

,Value,Time,Block
count,1.439050e+05,1.439050e+05,143905.000000
mean,1.384902e+20,1.439043e+09,54032.245224
std,3.003269e+21,3.821480e+06,28625.292713
min,0.000000e+00,0.000000e+00,0.000000
25%,5.000000e+18,1.438653e+09,30123.000000
50%,5.000000e+18,1.439091e+09,56685.000000
75%,5.000000e+18,1.439458e+09,78709.000000
max,3.232034e+23,1.439799e+09,100000.000000


In [6]:
df.columns

Index(['From', 'To', 'Value', 'Time', 'Block'], dtype='object')

In [20]:
x = df.groupby('From').count()['To']
y = df.groupby('To').count()['From']

In [19]:
df[df['From']=='0004b15be8cb573a7be023e203e268dbcffd302b']

,From,To,Value,Time,Block
111921,0004b15be8cb573a7be023e203e268dbcffd302b,2102cc711d03dfcedf9a443b9bd540dacbff0426,4.890000e+20,1439639772,90419
111929,0004b15be8cb573a7be023e203e268dbcffd302b,2102cc711d03dfcedf9a443b9bd540dacbff0426,1.000000e+18,1439639635,90413
116467,0004b15be8cb573a7be023e203e268dbcffd302b,2102cc711d03dfcedf9a443b9bd540dacbff0426,9.500000e+20,1439697931,93795


In [22]:
z = pd.concat([x,y])

In [25]:
x.count()

5789

In [27]:
df[df['To']=='0004b15be8cb573a7be023e203e268dbcffd302b']

,From,To,Value,Time,Block
57916,3e0b8ed86ed669e12723af7572fbacfe829b1e16,0004b15be8cb573a7be023e203e268dbcffd302b,1.490000e+21,1439101654,57346
57928,3e0b8ed86ed669e12723af7572fbacfe829b1e16,0004b15be8cb573a7be023e203e268dbcffd302b,2.000000e+17,1439101528,57336


In [47]:
x

From
0004b15be8cb573a7be023e203e268dbcffd302b      3
000df160997ba9420e4622996b05447871848834     13
0015b5e5afe7eba613cd1ba0a7345ed150adf5dc      1
0025bc2515c073784222cbcd1e988dba9a0f75af      2
0028795f5139935918a4176c80b01d32549e2819      4
0037a6b811ffeb6e072da21179d11b1406371c63      3
0037ce3d4b7f8729c8607d8d0248252be68202c0     17
003a0659d5452a076403d1411e617c9665955d59      5
003a6cefcd4377033674d8a80ddbfb3596bd13dd      3
0043bc318929d9146864870b0988d10f600f1090      1
004b37d3e250eb1826fc6a587deddc18e7be6f6f      1
004bfbe1546bc6c65b5c7eaa55304b38bbfec6d3      3
0053a170b33712997013623b9e5a27198e9bc9f9      2
007622d84a234bb8b078230fcf84b67ae9a8acae      2
007695872ab13e1016138e7bf2be234ca7a77740      1
007761bfef91f6d2b5aa41b0197bf496978eeb73      1
007a496da7690601206bc3c67176bb675ffb9b7b      1
007f7f58d3eb5b7510a301ecc749fc1fcddbe14d    156
00818287ff6ba78a461d21bd5832891592dc6c6b      7
0089508679abf8c71bf6781687120e3e6a84584d      7
008b1d771af0dcfd83c30d9d93f3b83cdf1

In [8]:
Web3.isAddress('fee4deb01301a2633e0046305a31fca8d04f416c')

True

In [18]:
add = Web3.toChecksumAddress('ffcd9cae6d9f54fd8ae05296f839892677ff6eaa')

In [19]:
len(w3.eth.getCode(add))

0

In [20]:
def isAccount(address):
    cs_add = Web3.toChecksumAddress(address)
    if len(w3.eth.getCode(cs_add))==0:
        return True
    return False

In [4]:
import numpy as np